In [2]:
import pathlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import guitarpro as pygp
import glob

path_to_general_read_folder = "..\..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\..\data\BGTG\BGTG-v0.1"

In [2]:
df_rg = pd.read_csv('../../data/df_rg_1.csv')

In [3]:
df_rg.head()

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation
0,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07
1,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,2,4.580104e-07
2,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,3,6.601997e-06
3,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,4,4.111985e-04
4,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,5,4.352387e-05


In [4]:
df_inst_part = df_rg.drop_duplicates(subset=['Dadagp_Path', 'Partie'])
# This removes the measures, conserving only one row for each part of each song
df_inst_part.shape[0], df_rg.shape[0], df_inst_part['Dadagp_Path'].nunique()
# Total number of parts, total number of rows, total number of songs

(56419, 4164431, 24051)

In [7]:
total_df_track = pd.DataFrame()

# For each track we add the DadaGP_name column
for path in tqdm(df_inst_part['Dadagp_Path'].unique()):
    df_track = df_inst_part[df_inst_part['Dadagp_Path'] == path].copy() # We copy to avoid SettingWithCopyWarning
    
    dada_gp_path = df_track['Dadagp_Path'].values[0]
    
    try:
        gp_file = pygp.parse(dada_gp_path)
        
    except:
        continue
    
    # No need for this anymore normally as the path are retrieved from DadaGP
    # try:
    #     gp_file = pygp.parse(dada_gp_path)
    # except:
    #     print('Error with file:', dada_gp_path)
    #     continue
        
    inst_list = []
    for part in gp_file.tracks:
        part_name = part.name
        
        if part_name not in df_track['Partie'].values:
            continue
        inst_name = df_track[df_track['Partie'] == part_name]['Instrument'].values[0]
        inst_list.append((part_name, inst_name))
    
    # Create the Dadagp_Name column filled with 'not_filled_yet'
    df_track['Dadagp_Name'] = 'not_filled_yet'
    nb_clean = 0
    nb_distorted = 0
    for part_inst_tuple in inst_list:
        part = part_inst_tuple[0]
        inst = part_inst_tuple[1]
        if inst == 'clean':
            df_track.loc[df_track['Partie'] == part, 'Dadagp_Name'] = 'clean' + str(nb_clean)
            nb_clean += 1
        elif inst == 'distorted':
            df_track.loc[df_track['Partie'] == part, 'Dadagp_Name'] = 'distorted' + str(nb_distorted)
            nb_distorted += 1
        else:
            df_track.loc[df_track['Partie'] == part, 'Dadagp_Name'] = inst
            
    # if 'not_filled_yet' in df_track['Dadagp_Name'].unique():
        # display(df_track)
        # print(inst_list)
    
    # Join df_track with df_inst_part
    total_df_track = pd.concat([total_df_track, df_track])

df_inst_part.shape, total_df_track.shape

# The method is correct but takes a long time to compute (1h)

  0%|          | 0/24051 [00:00<?, ?it/s]

100%|██████████| 24051/24051 [3:14:57<00:00,  2.06it/s]      


((56419, 8), (55793, 9))

In [33]:
path_to_test = total_df_track[total_df_track['File_Name'] == '311 - Homebrew (2)']['Dadagp_Path'].values

path_to_test

array(['..\\..\\data\\DadaGP\\DadaGP-v1.1\\3\\311\\311 - Homebrew (2).gp3.pygp.gp5',
       '..\\..\\data\\DadaGP\\DadaGP-v1.1\\3\\311\\311 - Homebrew (2).gp3.pygp.gp5',
       '..\\..\\data\\DadaGP\\DadaGP-v1.1\\3\\311\\311 - Homebrew (2).gp3.pygp.gp5',
       '..\\..\\data\\DadaGP\\DadaGP-v1.1\\3\\311\\311 - Homebrew (2).gp3.pygp.gp5',
       '..\\..\\data\\DadaGP\\DadaGP-v1.1\\3\\311\\311 - Homebrew (2).gp3.pygp.gp5'],
      dtype=object)

In [ ]:

gp_file = pygp.parse(path_to_test)
    
inst_list = []
for part in gp_file.tracks:
    part_name = part.name
    print(part_name)
    
    if part_name not in df_track['Partie'].values:
        continue
    inst_name = df_track[df_track['Partie'] == part_name]['Instrument'].values[0]
    inst_list.append((part_name, inst_name))

In [8]:
total_df_track['Dadagp_Name'].unique()
# We should not have any 'not filled yet'
# 'not filled yet' values correspond to songs where we lost the path.
# This happens when several songs have the same name but different extensions (gp3, gp4, gp5)
# In notebook 1 we simply take the first path of the name, which is not always the right one.
# As the computation necessary to assign the correct path is extremely long, we will simply drop these songs.

array(['distorted0', 'clean0', 'clean1', 'bass', 'distorted1',
       'distorted2', 'not_filled_yet', 'distorted3', 'clean2', 'clean3',
       'clean4', 'distorted4', 'distorted5'], dtype=object)

In [9]:
# Merge the DadaGP_name back

new_df_rg = df_rg.merge(total_df_track[['Dadagp_Path', 'Partie', 'Dadagp_Name']],
                                      on=['Dadagp_Path', 'Partie'], how='left')



In [ ]:
list_test = ['distorted0', 'clean0', 'clean1', 'bass', 'distorted1',
       'distorted2', 'distorted3', 'clean2', 'clean3', 'clean4',
       'distorted4', 'distorted5']

df_rg_2 = new_df_rg[new_df_rg['Dadagp_Name'].isin(list_test)]

df_rg_2['Dadagp_Name'].unique(), df_rg_2['File_Name'].nunique()

In [ ]:
# Some dupicated songs have not been removed using this method because they have the same name AND the same parts' names
# in the gp file. To remove them, we perform a drop_duplicates on the Dadagp_Name, Mesure and Dadagp_Path columns.
initial_shape = df_rg_2.shape[0]
print(df_rg_2.shape, df_rg_2['Dadagp_Path'].nunique())
df_rg_2 = df_rg_2.drop_duplicates(subset=['Dadagp_Name', 'Mesure', 'Dadagp_Path'])
print(df_rg_2.shape, df_rg_2['Dadagp_Path'].nunique())

lost_rows = initial_shape - df_rg_2.shape[0]
lost_rows

(4037501, 9) 23123
(4026353, 9) 23123


11148

In [8]:
df_rg_2.to_csv('../../data/df_rg_2.csv', index=False)